<a href="https://colab.research.google.com/github/fastai-energetic-engineering/ashrae/blob/master/ASHRAE_energy_fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ASHRAE Energy Prediction

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [fastai, kaggle]
- image: images/some_folder/your_image.png
- hide: false
- search_exclude: false


In [2]:
#collapse
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [3]:
#collapse
#from fastbook import *
import os
#from google.colab import files
import pandas as pd
import datetime

In [4]:
# Load ASHRAE energy dataset from your google drive
p = 'gdrive/MyDrive/Colab Notebooks/ashrae'
os.chdir(p)

## Joining Tables

Our training data comprised of three tables:
- `building_metadata.csv`
- `weather_train.csv`
- `train.csv`

We need to join the tables. First, let's see what's in the tables.

In [5]:
building = pd.read_csv('data/building_metadata.csv')
weather = pd.read_csv('data/weather_train.csv')
train = pd.read_csv('data/train.csv')

`building` contains the buildings' metadata.

In [6]:
building.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN


- `site_id` - Foreign key for the weather files.
- `building_id` - Foreign key for training.csv
- `primary_use` - Indicator of the primary category of activities for the building based on EnergyStar property type definitions
- `square_feet` - Gross floor area of the building
- `year_built` - Year building was opened
- `floor_count` - Number of floors of the building

`weather` contains weather data from the closest meteorological station.

In [7]:
weather.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


- `site_id`
- `air_temperature` - Degrees Celsius
- `cloud_coverage` - Portion of the sky covered in clouds, in oktas
- `dew_temperature` - Degrees Celsius
- `precip_depth_1_hr` - Millimeters
- `sea_level_pressure` - Millibar/hectopascals
- `wind_direction` - Compass direction (0-360)
- `wind_speed` - Meters per second

Finally, `train` contains the target variable, `meter reading`, which represents energy consumption in kWh.

In [8]:
train.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.0
1,1,0,2016-01-01 00:00:00,0.0
2,2,0,2016-01-01 00:00:00,0.0
3,3,0,2016-01-01 00:00:00,0.0
4,4,0,2016-01-01 00:00:00,0.0


- `building_id` - Foreign key for the building metadata.
- `meter` - The meter id code. Read as {0: electricity, 1: chilledwater, 2: steam, 3: hotwater}. Not every building has all meter types.
- `timestamp` - When the measurement was taken
- `meter_reading` - The target variable. Energy consumption in kWh (or equivalent).

Apparently there was some issues regarding the timestamps, as noted by [this post](https://www.kaggle.com/c/ashrae-energy-prediction/discussion/115040#latest-667889). The timestamp in the weather and meter reading table were in GMT and local time, respectively. We have to keep this in mind before merging the tables.

Here I wrote a function that can prepare train and test data accordingly.

In [9]:
def prepare_data(type='train'):
    assert type in ['train', 'test']
    
    # read data
    building = pd.read_csv('data/building_metadata.csv')
    weather = pd.read_csv(f'data/weather_{type}.csv')
    data = pd.read_csv(f'data/{type}.csv')

    # convert datetime
    data['timestamp'] = pd.to_datetime(data['timestamp'])

    # adjust timestamp
    timediff = {0:4,1:0,2:7,3:4,4:7,5:0,6:4,7:4,8:4,9:5,10:7,11:4,12:0,13:5,14:4,15:4}
    weather['time_diff']= weather['site_id'].map(timediff)
    weather['time_diff'] = weather['time_diff'].apply(lambda x: datetime.timedelta(hours=x))
    weather['timestamp_gmt'] = pd.to_datetime(weather['timestamp'])
    weather['timestamp'] = weather['timestamp_gmt'] - weather['time_diff']

    # merge table
    data = data.merge(building, on='building_id', how='left')
    data = data.merge(weather, on=['site_id','timestamp'], how='left')

    return data

Let's try this function out!

In [10]:
train = prepare_data('train')
train.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,time_diff,timestamp_gmt
0,0,0,2016-01-01,0.0,0,Education,7432,2008.0,NaN,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6,0 days 04:00:00,2016-01-01 04:00:00
1,1,0,2016-01-01,0.0,0,Education,2720,2004.0,NaN,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6,0 days 04:00:00,2016-01-01 04:00:00
2,2,0,2016-01-01,0.0,0,Education,5376,1991.0,NaN,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6,0 days 04:00:00,2016-01-01 04:00:00
3,3,0,2016-01-01,0.0,0,Education,23685,2002.0,NaN,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6,0 days 04:00:00,2016-01-01 04:00:00
4,4,0,2016-01-01,0.0,0,Education,116607,1975.0,NaN,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6,0 days 04:00:00,2016-01-01 04:00:00


That's it! In the next blogpost, I will show how to load this data into FastAI's `dataloaders`.

In [12]:
from fastai.tabular.all import *
cat_names = ['building_id', 'meter', 'site_id', 'primary_use']
#cont_names = ['timestamp', 'square_feet', 'year_built', 'floor_count', 'air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed', 'time_diff', 'timestamp_gmt']
cont_names = ['square_feet', 'year_built', 'floor_count', 'air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed']
procs = [Categorify, FillMissing, Normalize]
dls = TabularDataLoaders.from_df(train.head(64*64), procs=procs, cat_names=cat_names, cont_names=cont_names, 
                                 y_names="meter_reading")

In [13]:
dls.show_batch()
# Why do continuous variables show as boolean!!!

,building_id,meter,site_id,primary_use,year_built_na,floor_count_na,air_temperature_na,cloud_coverage_na,dew_temperature_na,precip_depth_1_hr_na,sea_level_pressure_na,wind_direction_na,wind_speed_na,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,meter_reading
0,1074,0,13,Office,True,True,False,True,False,False,False,False,False,89858.000806,1969.000000,3.0,-7.2,1.495093e-08,-10.6,-3.009280e-10,1022.799988,270.000001,5.1,39.249001
1,1347,1,15,Education,False,True,True,True,True,True,True,True,True,322340.995825,1995.000001,3.0,7.2,1.495093e-08,-1.5,-3.009280e-10,1021.500000,250.000001,3.1,26.539600
2,1302,0,14,Office,True,True,False,True,False,False,False,False,False,90411.000663,1969.000000,3.0,5.0,1.495093e-08,-2.2,-3.009280e-10,1019.200012,290.000000,3.1,115.796997
3,456,0,3,Public services,True,True,False,True,False,False,False,False,False,14124.997087,1969.000000,3.0,7.8,1.495093e-08,0.6,-3.009280e-10,1021.799988,340.000004,4.1,15.240000
4,958,0,9,Entertainment/public assembly,True,True,False,True,False,False,False,False,False,234818.996784,1969.000000,3.0,9.4,1.495093e-08,3.3,-3.009280e-10,1028.699951,20.000008,4.1,211.000000
5,880,1,9,Education,True,True,False,True,False,False,False,False,False,157723.001572,1969.000000,3.0,9.4,1.495093e-08,3.9,-3.009280e-10,1029.500000,9.999996,4.6,0.000000
6,907,0,9,Entertainment/public assembly,True,True,False,True,False,False,False,False,False,44563.997473,1969.000000,3.0,9.4,1.495093e-08,3.9,-3.009280e-10,1029.500000,9.999996,4.6,0.000000
7,1148,0,13,Office,True,True,False,False,False,False,False,False,False,861523.983697,1969.000000,3.0,-7.2,8.000000e+00,-10.6,-3.009280e-10,1022.900024,279.999996,5.7,806.411987
8,928,1,9,Education,True,True,False,True,False,False,False,False,False,179115.003653,1969.000000,3.0,9.4,1.495093e-08,3.9,-3.009280e-10,1029.500000,9.999996,4.6,0.000000
9,1098,0,13,Office,True,True,False,False,False,False,False,False,False,131745.000830,1969.000000,3.0,-7.2,8.000000e+00,-10.6,-3.009280e-10,1022.900024,279.999996,5.7,28.898001


In [14]:
learn = tabular_learner(dls, metrics=accuracy)
learn.fit_one_cycle(10)

epoch,train_loss,valid_loss,accuracy,time
0,6670558720.000000,2686055.250000,0.233211,00:00
1,6678545920.000000,2684323.750000,0.233211,00:00
2,4848667136.000000,2683248.500000,0.233211,00:00
3,4879524864.000000,2681057.750000,0.233211,00:00
4,7189390848.000000,2681627.500000,0.233211,00:00
5,6950934016.000000,2679952.000000,0.233211,00:00
6,7168902656.000000,2680648.000000,0.233211,00:00
7,5691739648.000000,2679432.000000,0.233211,00:00
8,7024227328.000000,2680359.750000,0.233211,00:00
9,5814515200.000000,2679705.750000,0.233211,00:00


In [29]:
learn.show_results()

,building_id,meter,site_id,primary_use,year_built_na,floor_count_na,air_temperature_na,cloud_coverage_na,dew_temperature_na,precip_depth_1_hr_na,sea_level_pressure_na,wind_direction_na,wind_speed_na,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,meter_reading,meter_reading_pred
0,0.0,4.0,3.0,3.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.601793,1.725681,-0.165854,-0.114991,-0.446542,-0.894673,0.337945,-1.232145,-1.375018,-1.347227,2.930700,0.531586
1,0.0,1.0,6.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,-0.263854,0.013044,0.255363,-0.114991,-0.446542,-0.030036,0.337945,0.407329,0.356379,-0.012074,33.599998,0.244483
2,0.0,1.0,4.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,-0.460186,0.013044,-0.165854,0.010970,-0.446542,-0.030036,0.337945,1.003510,1.077795,1.259501,17.250000,-0.759724
3,0.0,1.0,4.0,7.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,-0.280611,0.013044,-0.165854,0.010970,-0.446542,-0.030036,0.337945,1.003510,1.077795,1.259501,55.930000,-0.583110
4,0.0,1.0,4.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,-0.289677,0.013044,-0.165854,0.010970,-0.446542,-0.030036,0.337945,1.003510,1.077795,1.259501,33.450001,-0.589202
5,624.0,1.0,7.0,7.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.213197,0.013044,-0.165854,0.010970,2.239426,0.430174,0.337945,0.407329,-1.014310,-0.012074,201.082993,2.153915
6,0.0,1.0,1.0,12.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.575720,1.088421,-0.165854,2.572169,2.239426,2.675441,-2.959075,0.109254,0.428521,0.305820,0.000000,-1.731423
7,0.0,1.0,6.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,-0.697086,-1.978393,-1.429505,-0.114991,-0.446542,-0.030036,0.337945,0.407329,0.356379,-0.012074,1.700000,-1.049081
8,0.0,1.0,4.0,10.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,-0.694196,0.013044,-0.165854,0.010970,-0.446542,-0.030036,0.337945,1.003510,1.077795,1.259501,15.240000,-0.987957


To do:


1.   Figure out why continuous variables are shown as boolean
2.   Handle dates
3.   Create training and validation sets, rather than using TabularDataLoader

